In [ ]:
import os
from glob import glob
import numpy as np

from models.VariationalAutoEncoder import VariationalAutoEncoder
from keras.preprocessing.image import ImageDataGenerator

## Set Run Parameters

In [ ]:
# run params
SECTION = 'vae'
RUN_ID = '0001'
DATA_NAME = 'faces'
if not os.path.exists("run"):
    os.mkdir("run")
if not os.path.exists(f"run/{SECTION}"):
    os.mkdir(f"run/{SECTION}")
RUN_FOLDER = f'run/{SECTION}/'
RUN_FOLDER += '_'.join([RUN_ID, DATA_NAME])

if not os.path.exists(RUN_FOLDER):
    os.mkdir(RUN_FOLDER)
    os.mkdir(os.path.join(RUN_FOLDER, 'viz'))
    os.mkdir(os.path.join(RUN_FOLDER, 'images'))
    os.mkdir(os.path.join(RUN_FOLDER, 'weights'))

MODE =  'build' #'load' #

DATA_FOLDER = './data/celeb/'

## Load the data

In [ ]:
input_dim = (128, 128, 3)
batch_size = 32
reconstruction_loss_multiplier = 10000

In [ ]:
filenames = np.array(glob(os.path.join(DATA_FOLDER, '*/*.jpg')))

NUM_IMAGES = len(filenames)

In [ ]:
data_gen = ImageDataGenerator(rescale=1./255)

data_flow = data_gen.flow_from_directory(DATA_FOLDER, 
                                         target_size = input_dim[:2], 
                                         batch_size = batch_size, 
                                         shuffle = True, 
                                         class_mode = 'input', 
                                         subset = "training")

## Define the structure of the neural network

In [ ]:
encoder_architecture = [
    {'filter': 32, 'kernel': (3, 3), 'stride': 2},
    {'filter': 64, 'kernel': (3, 3), 'stride': 2},
    {'filter': 64, 'kernel': (3, 3), 'stride': 2},
    {'filter': 64, 'kernel': (3, 3), 'stride': 2},
    ]
decoder_architecture = [
    {'filter': 64, 'kernel': (3, 3), 'stride': 2},
    {'filter': 64, 'kernel': (3, 3), 'stride': 2},
    {'filter': 32, 'kernel': (3, 3), 'stride': 2},
    {'filter': 3, 'kernel': (3, 3), 'stride': 2},
    ]
latent_dim = 200
use_batch_norm = True
use_dropout = True

In [ ]:
ae = VariationalAutoEncoder(
    input_dim=input_dim,
    latent_dim=latent_dim,
    encoder_params=encoder_architecture,
    decoder_params=decoder_architecture,
    use_batch_norm=use_batch_norm,
    use_dropout=use_dropout,
    reconstruction_loss_multiplier=reconstruction_loss_multiplier,
)

In [ ]:
if MODE == 'build':
    ae.save(RUN_FOLDER)
else:
    ae.load_weights(os.path.join(RUN_FOLDER, 'weights/weights.h5'))

In [ ]:
ae.autoencoder_model.summary()

In [ ]:
ae.encoder_model.summary()

In [ ]:
ae.decoder_model.summary()

## Train the autoencoder

In [ ]:
LEARNING_RATE = 0.0005
BATCH_SIZE = 32
INITIAL_EPOCH = 0
TOTAL_EPOCHS = 200
PRINT_EVERY_N_BATCHES = 100


In [ ]:
ae.compile(LEARNING_RATE)

In [ ]:
ae.train_with_generator(     
    data_flow
    , epochs = TOTAL_EPOCHS
    , steps_per_epoch = NUM_IMAGES / BATCH_SIZE
    , run_folder = RUN_FOLDER
    , print_every_n_batches = PRINT_EVERY_N_BATCHES
    , initial_epoch = INITIAL_EPOCH
)